In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

# Data Loading & Clearing

In [2]:
#Ideas

#How could user enter book: Enter part of title, then search in books dataframe with .title.str.contains(<input>), 
#then sort results by number of ratings to get the most rated book and hence the highest chance of good recommendations.

#Also, filter the outputs not to contain the same string which was given as input

In [3]:
ratings = pd.read_csv('BX-Book-Ratings.csv', sep = ';' , encoding='utf-8', on_bad_lines='warn', encoding_errors = 'ignore') #There are some suspicious ISBNs, we might loos them as they will not match with books on merging
ratings.rename(columns={'User-ID':'user'
                        , 'Book-Rating': 'rating'}, inplace=True)
ratings.shape
ratings.nunique()

(1149780, 3)

user      105283
ISBN      340553
rating        11
dtype: int64

In [4]:
# dtype_mapping = {
#     'Year-Of-Publication': float
# }
books = pd.read_csv('BX-Books.csv', sep = ';', encoding='utf-8', on_bad_lines='warn', encoding_errors = 'ignore')

#books.drop(books[books['Unnamed: 8'].notna()].index, axis = 0, inplace=True) #Dropping the shifted rows. There is 19 of them, not worth the manual correction I think.
#books.drop(['Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'], axis = 1, inplace=True)
books.rename(columns={'Book-Title':'title'
                        , 'Book-Author': 'author'
                        , 'Year-Of-Publication': 'year'
                        , 'Publisher': 'publisher'}, inplace=True)
#books
books.nunique()

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
d:\Program_files\Anaconda_nw_install_2022\lib\site-packages\IPython\core\i

ISBN           271360
title          242135
author         102023
year              202
publisher       16807
Image-URL-S    271044
Image-URL-M    271044
Image-URL-L    271041
dtype: int64

In [5]:
users = pd.read_csv('BX-Users.csv', sep = ';' , encoding='utf-8', on_bad_lines='warn', encoding_errors = 'ignore')
users.rename(columns={'User-ID':'user'
                        , 'Location': 'location'
                        , 'Age': 'age'}, inplace=True)
print(users.shape)
users.nunique()

(278858, 3)


user        278858
location     57293
age            165
dtype: int64

In [6]:
ratings

,user,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [65]:
#how many ratings each book have?
explicit_rating_counts = ratings[ratings['rating'] > 0][['ISBN', 'rating']].groupby('ISBN', as_index=False).count().rename(columns={'rating':'explicit_ratings'})
implicit_rating_counts = ratings[ratings['rating'] == 0][['ISBN', 'rating']].groupby('ISBN', as_index=False).count().rename(columns={'rating':'implicit_ratings'})
#implicit_rating_counts

if 'explicit_ratings' not in books.columns:
    books = books.merge(explicit_rating_counts, on = 'ISBN', how='left')
    books = books.merge(implicit_rating_counts, on = 'ISBN', how='left')

In [8]:
books.year.unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

In [9]:
#Clearing the years of publication
books.drop(books[books['year'] == 'Gallimard'].index, axis = 0, inplace=True)
books.drop(books[books['year'] == 'DK Publishing Inc'].index, axis = 0, inplace=True)
books['year'] = books['year'].astype(int, errors='ignore')

#books.loc[books.year == 0, 'year'] = np.NAN ## <<------- this forces the column to be floats. So I will let the zeros stay for now, see how it will affect the rest.

In [10]:
books.year.unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984,    0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, 2012, 2006, 1909, 2008, 1378,
       1919, 1922, 1897, 2024, 1376, 2037])

In [11]:
#books['year'] = books['year'].astype(int, errors='ignore')
books['year'].median()

1995.0

In [12]:
#Adding a time period when the book was released, instead of a year
#The idea here is to add firslty the tens of the book release, and secondly the most adjecent tens as a secondary variable
#Hence e.g. for 1991 -> 1990 and 1980, and for 1998 -> 1990 and 2000
books['period1'] = (books.year // 10) * 10
books['period2'] = books['period1'] + ((((books.year - books.period1) >= 5)*2-1)*10)


In [13]:
#books[['ISBN', 'year', 'period1', 'period2']]

# Similarity Selection

In [95]:
##Inputs/argumets
# input_name = 'The Lord of the Rings' #Unclear input, when name of the series is given, instead of a name of a book
input_name = 'The Fellowship of the Ring'
# input_name = 'The Eye of the World'
# input_name = 'Great Expectations' 
# input_name = 'The Color of Magic'
# input_name = 'The Egyptian' #Here is a problem that title contains 'A Novel'. Giving more weight to author could help?
# input_name = 'Hotel New Hampshire'
# input_name = 'The Shining'
# input_name = 'For Whom the Bell Tolls'

input_book = (books[(books['title'].str.contains(input_name))
                .replace(np.NaN, False)]
                .sort_values(['explicit_ratings', 'implicit_ratings'], ascending = False).head(1)[['ISBN', 'title', 'author', 'year', 'publisher', 
                                                                                                   'explicit_ratings', 'implicit_ratings',
                                                                                                    'period1', 'period2']]
                )

display(input_book)

input_index = input_book.index[0]
input_ISBN = input_book.ISBN.values[0]
              
choose_n = 5

,ISBN,title,author,year,publisher,explicit_ratings,implicit_ratings,period1,period2
1571,0345339703,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,1986,Del Rey,131.0,126.0,1980,1990


In [98]:
#books[(books['author'].str.contains('Tolkien')).replace(np.NaN, False)]
#books[(books['author'].str.contains('Robert Jordan')).replace(np.NaN, False)].head(10)
#books[(books['title'].str.contains('The Eye of the World')).replace(np.NaN, False)]

In [99]:
books['info'] = ( books.title + ' ' +
                 (books.author.fillna('') + ' ')*1 
                 + books.publisher.fillna('') + ' ' 
                 + books.year.astype(str) + ' ' 
                 + books.period1.astype(str) + ' ' 
                 + books.period2.astype(str))
#Here possibly add weight by multiplying the number of times the info appears?
#But if I will use cosine similarity, this way of adding weights is probably useless (?): https://www.pinecone.io/learn/vector-similarity/

In [100]:
books[books['info'].isna()] #empty df
for i in books['info'][0:3]:
    print(i)

,ISBN,title,author,year,publisher,Image-URL-S,Image-URL-M,Image-URL-L,explicit_ratings,implicit_ratings,period1,period2,info


Classical Mythology Mark P. O. Morford Oxford University Press 2002 2000 1990
Clara Callan Richard Bruce Wright HarperFlamingo Canada 2001 2000 1990
Decision in Normandy Carlo D'Este HarperPerennial 1991 1990 1980


In [101]:
vectorizer = CountVectorizer()
term_matrix = vectorizer.fit_transform(books['info'])

In [102]:
len(books)
term_matrix.shape
#print(term_matrix[0:5])

271357

(271357, 117284)

In [103]:
#similarities = cosine_similarity(term_matrix, dense_output=False)

In [104]:
#Computing similarities of all books takes too long. Perhaps I could select one book and compute similarity of it with the remaining.

In [105]:
row_to_compare = term_matrix[input_index, :]
similarities = cosine_similarity(row_to_compare, term_matrix)

In [106]:
similarities
similarities.shape

#print(np.sort(similarities)[:,-10:-1])
#print(similarities.sort())

indices_of_largest = np.argsort(similarities)[:, -(100+1):-1]
indices_of_largest = np.flip(indices_of_largest.flatten())
#indices_of_largest

array([[0.05679618, 0.05679618, 0.11973687, ..., 0.53881591, 0.05679618,
        0.27322953]])

(1, 271357)

In [107]:
similarity_selection = books.iloc[indices_of_largest.flatten()].copy()
similarity_selection[~(similarity_selection.title.str.contains(input_name))].iloc[0:choose_n]

,ISBN,title,author,year,publisher,Image-URL-S,Image-URL-M,Image-URL-L,explicit_ratings,implicit_ratings,period1,period2,info
4206,0345339738,"The Return of the King (The Lord of the Rings,...",J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339738.0...,http://images.amazon.com/images/P/0345339738.0...,http://images.amazon.com/images/P/0345339738.0...,77.0,96.0,1980,1990,"The Return of the King (The Lord of the Rings,..."
780,0345339711,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339711.0...,http://images.amazon.com/images/P/0345339711.0...,http://images.amazon.com/images/P/0345339711.0...,83.0,94.0,1980,1990,"The Two Towers (The Lord of the Rings, Part 2)..."
6866,0345340426,The Hobbit and The Lord of the Rings,J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345340426.0...,http://images.amazon.com/images/P/0345340426.0...,http://images.amazon.com/images/P/0345340426.0...,6.0,2.0,1980,1990,The Hobbit and The Lord of the Rings J.R.R. TO...
3354,0345339681,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339681.0...,http://images.amazon.com/images/P/0345339681.0...,http://images.amazon.com/images/P/0345339681.0...,161.0,120.0,1980,1990,The Hobbit : The Enchanting Prelude to The Lor...
135872,039556008X,The War of the Ring (The History of The Lord o...,J.R.R. Tolkien,1990,Houghton Mifflin,http://images.amazon.com/images/P/039556008X.0...,http://images.amazon.com/images/P/039556008X.0...,http://images.amazon.com/images/P/039556008X.0...,1.0,NaN,1990,1980,The War of the Ring (The History of The Lord o...


# Explicit Ratings Selection

In [108]:
explicit_ratings = ratings[ratings.rating != 0].copy()
implicit_ratings = ratings[ratings.rating == 0].copy()
implicit_ratings.rating =+ 1

In [114]:
#Converting explicit ratings into pivotted form in a sparse matrix (pd.pivot() cannot produce sparse it seems)
#kudos to this post: https://stackoverflow.com/questions/31661604/efficiently-create-sparse-pivot-tables-in-pandas
ISBN_c = CategoricalDtype(explicit_ratings.ISBN.unique(), ordered=False)
user_c = CategoricalDtype(explicit_ratings.user.unique(), ordered=False)

row = explicit_ratings.ISBN.astype(ISBN_c).cat.codes
col = explicit_ratings.user.astype(user_c).cat.codes
explicit_sparse_matrix = csr_matrix((explicit_ratings["rating"], (row, col)), 
                           shape=(ISBN_c.categories.size, user_c.categories.size))

sparse_matrix

#This will be likely unnecessary in the end
dfs = pd.DataFrame.sparse.from_spmatrix(explicit_sparse_matrix, 
                         index=ISBN_c.categories, 
                         columns=user_c.categories
                         )
dfs.shape

<185972x77805 sparse matrix of type '<class 'numpy.int64'>'
	with 433671 stored elements in Compressed Sparse Row format>

(185972, 77805)

In [118]:
ISBN_c.categories #Either get index of input ISBN from the sparse matrix and use sparse matrix, or use the df and use ISBN. 
#using sparse matrix seems better to me, will see what is fatser in knn

Index(['0155061224', '052165615X', '0521795028', '3257224281', '0600570967',
       '038550120X', '342310538', '0060517794', '0671537458', '0679776818',
       ...
       '1931333246', '0316185922', '0312264186', '0345377818', '0425156737',
       '0671563149', '1575660792', '0380796155', '0806917695', '05162443314'],
      dtype='object', length=185972)

In [113]:
dfs.loc[input_ISBN, :]
(dfs.loc[input_ISBN, :] >0).sum()

276726    0
276729    0
276736    0
276737    0
276744    0
         ..
276688    0
276704    0
276709    0
276721    0
276723    0
Name: 0345339703, Length: 77805, dtype: Sparse[int64, 0]

131

In [111]:
(dfs.iloc[input_index] >0).sum()
explicit_sparse_matrix[input_index, :]

15

<1x77805 sparse matrix of type '<class 'numpy.int64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [ ]:
#KNN
